In [ ]:
%env USING_RUN True

%run ./greenhouse/greenhouse.ipynb import Greenhouse

In [ ]:
def run_simulation(greenhouse, date_range, resample_period):
    time_period = pd.to_timedelta(resample_period).total_seconds()

    df = get_weather_data(date_from=date_range[0], date_to=date_range[1], resample_period=resample_period)

    for timestamp, row in df.iterrows():
        try:
            results = greenhouse.run(timestamp, row)
        except Exception as e:
            traceback.print_exc()
            break

        for col in results.keys():
            df.loc[timestamp, col] = results[col]

    return df

In [ ]:
resample_period="15min"

configs = {
    "baseline": Greenhouse(resample_period=resample_period, block_sunlight=True, airflow_mode="CONST:0"),
    "solarbrite": Greenhouse(resample_period=resample_period, roof_panel_type="solarbrite"),
    "insolight": Greenhouse(resample_period=resample_period, roof_panel_type="insolight"),
    "polycarbonate": Greenhouse(resample_period=resample_period, roof_panel_type="polycarbonate"),
}

dates = {
    "winter": ["2020-01-01", "2020-01-02"],
    "spring": ["2020-03-01", "2020-03-02"],
    "summer": ["2020-06-01", "2020-06-02"],
    "fall": ["2020-09-01", "2020-09-02"]
}

dfs = {}
results = []
for config_type in configs:
    dfs[config_type] = {}
    greenhouse = configs[config_type]

    results_of_date = {}
    for date_type in dates:
        date_range = dates[date_type]

        df = run_simulation(greenhouse, date_range, resample_period)

        total_cost = get_total_cost(df, greenhouse)

        dfs[config_type][date_type] = df
        print(f"Simulation done for {config_type} - {date_type}")
        results_of_date[f"cost_{date_type}"] = round(total_cost, 2)

    results.append({
        "config_type": config_type,
        **results_of_date
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df["total_cost"] = round(results_df[[f"cost_{date_type}" for date_type in dates]].sum(axis=1), 2)
results_df

In [ ]:
plot_in_grid(dfs, "block_sunlight", plot_energy)

In [ ]:
plot_in_grid(dfs, "with_sunlight", plot_energy)

In [ ]:
plot_in_grid(dfs, "3", plot_energy)

In [ ]:
plot_in_grid(dfs, "4", plot_energy)

In [ ]:
plot_in_grid(dfs, "max_temp_18", plot_temperature)

In [ ]:
plot_in_grid(dfs, "max_temp_18", plot_humidity)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>